In [1]:
# Python script to do the analysis based on the account table
import openpyxl
import numpy as np
import pandas as pd
import os
from util import rs_conn, mail
import time
import datetime
import logging

In [2]:
# Get the redshift user name and password
rs_user = os.getenv('RS_USER')
rs_pass = os.getenv('RS_PASS')

# Connect to the Redshift
rs = rs_conn.RS_CONN(rs_user=rs_user, rs_pass=rs_pass)
rs.set_Conn()

# Get query data
query = open(f'./sql_queries/overview_data.sql', 'r').read()
df = rs.get_result_df(sql_query=query)
rs.conn.close()

In [5]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
end_month_date,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,...,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31
issued_card_cnt,3602,11297,15678,18221,20078,21547,23673,25988,29991,41970,...,1039979,1158454,1283130,1386236,1478037,1584884,1665970,1730185,1787695,1823588
delivery_cnt,3533,11067,15368,17870,19681,21129,23231,25511,29470,41208,...,1028431,1143792,1265191,1366075,1456271,1560940,1640280,1701420,1754687,1782087
activated_acct_cnt,919,4832,11782,15674,17829,19275,21728,24093,26819,36080,...,977365,1094522,1213207,1328654,1431647,1527057,1615083,1662908,1732330,1784156


In [14]:
df_empty = pd.DataFrame()

In [15]:
result = pd.concat([df_empty, df.T], axis=0, join='outer', ignore_index=False)

In [16]:
result

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
end_month_date,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,...,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31
issued_card_cnt,3602,11297,15678,18221,20078,21547,23673,25988,29991,41970,...,1039979,1158454,1283130,1386236,1478037,1584884,1665970,1730185,1787695,1823588
delivery_cnt,3533,11067,15368,17870,19681,21129,23231,25511,29470,41208,...,1028431,1143792,1265191,1366075,1456271,1560940,1640280,1701420,1754687,1782087
activated_acct_cnt,919,4832,11782,15674,17829,19275,21728,24093,26819,36080,...,977365,1094522,1213207,1328654,1431647,1527057,1615083,1662908,1732330,1784156


In [38]:
path = os.getcwd() + '/sql_queries'
all_files = os.listdir(path)
sql_files = list(filter(lambda f: f.endswith("summary.sql"), all_files))

In [45]:
tab_name = [x.split('.')[0] for x in sql_files]
tabs_dict = dict(('' + str(i), pd.DataFrame()) for i in tab_name)

In [55]:
tab_name

['usage_summary', 'revenue_summary', 'overview_summary', 'dq_summary']

In [53]:
# Get the redshift user name and password
rs_user = os.getenv('RS_USER')
rs_pass = os.getenv('RS_PASS')

# Connect to the Redshift
rs = rs_conn.RS_CONN(rs_user=rs_user, rs_pass=rs_pass)
rs.set_Conn()

# Get query data
for i,j in zip(tabs_dict.keys(), sql_files):
    query = open(f'./sql_queries/{j}', 'r').read()
    df = rs.get_result_df(sql_query=query).T
    tabs_dict[i] = df
rs.conn.close()

In [54]:
tabs_dict

{'usage_summary':                               0           1           2            3   \
 cycle_end_dt          2020-01-13  2020-02-12  2020-03-12   2020-04-13   
 actived_acct_cnt             110        1852        8384        13694   
 attrition_cnt                  0           0           2           14   
 co_cnt                         0           0           0            0   
 other_close_cnt                0           0           0            0   
 open_acct_cnt                110        1852        8382        13680   
 actived_credit_limit    454500.0   3898500.0  14803500.0   25356000.0   
 open_credit_limit       454500.0   3898500.0  14803500.0   25356000.0   
 active_credit_limit     350500.0   3102500.0  11204500.0   21221000.0   
 active_acct_cnt               74        1471        6562        11949   
 transactor_acct_cnt           74        1468        6399        10716   
 revolver_acct_cnt              0           0           0         3096   
 tot_end_os_bal_amt  